In [1]:
import pandas as pd
import scanpy as sc
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import random
from scvi.model import SCVI

/home/tlanser/miniconda3/envs/scVI/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/tlanser/miniconda3/envs/scVI/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/tlanser/miniconda3/envs/scVI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import scvi
import scanpy as sc

from matplotlib.pyplot import rc_context
sc.set_figure_params(dpi=100)

In [3]:
adata = sc.read_h5ad('../../h5ad/h5seurat_myeloid_BP2_unintegrated_annotated2.h5ad') #raw data

In [9]:
adata.obs['prog']

GEM1_AAACCCAAGACCGCCT-1             HC
GEM1_AAACCCAAGAGTGGCT-1             HC
GEM1_AAACCCAAGTATCCTG-1             HC
GEM1_AAACCCAAGTGAGGTC-1             HC
GEM1_AAACCCACAACGCATT-1             HC
                               ...    
GEM30_TTTGGTTTCACCCTTG-1    progressor
GEM30_TTTGGTTTCCGATTAG-1    progressor
GEM30_TTTGGTTTCGTTCAGA-1    progressor
GEM30_TTTGTTGAGCTCTTCC-1    progressor
GEM30_TTTGTTGTCCGGTAGC-1    progressor
Name: prog, Length: 382320, dtype: object

In [ ]:
# Annotate your AnnData object
adata.obs['BMI'] = bmi_values  # Continuous variable
adata.obs['disease_state'] = disease_states  # Categorical variable

# Train scVI
scvi_model = SCVI(adata)
scvi_model.train()

# Extract latent embeddings
latent_features = scvi_model.get_latent_representation()

In [ ]:
# Annotate your AnnData object
adata.obs['BMI'] = bmi_values  # Continuous variable
adata.obs['disease_state'] = disease_states  # Categorical variable

# Train scVI
scvi_model = SCVI(adata)
scvi_model.train()

# Extract latent embeddings
latent_features = scvi_model.get_latent_representation()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split data by disease state
for state in ['healthy', 'RRMS', 'progressors']:
    subset = adata[adata.obs['disease_state'] == state]
    latent = scvi_model.get_latent_representation(subset)
    bmi = subset.obs['BMI']

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(latent, bmi, test_size=0.2, random_state=42)

    # Train random forest regressor
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Feature importance
    importance = rf.feature_importances_

    # Evaluate
    predictions = rf.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"{state} MSE: {mse}")
    print(f"Top features for {state}: {importance.argsort()[-10:]}")
